In [ ]:
import pandas as pd
import numpy as np

# --- Step 1: Preparation - Review and Load Analyzed Data ---
# This section ensures the data is loaded, cleaned, and calculated
# exactly as we did in the previous analysis steps.

# Load the dataset
try:
    df = pd.read_csv('BCG.csv')
    print("Data loaded successfully!")
except FileNotFoundError:
    print("Error: 'BCG - Sheet1.csv' not found. Please make sure the file is in the correct directory.")
    print("Exiting. Please place the CSV file in the same directory and try again.")
    exit() # Exit the script if file is not found

# Define financial columns based on actual CSV headers from previous analysis
financial_cols = [
    'Total Revenue',
    'Net Income',
    'Total Assets',
    'Total Libalities', # Corrected spelling
    'Cash Flow from operating activities' # Corrected capitalization/spacing
]

# Convert financial columns to numeric, coercing errors to NaN
for col in financial_cols:
    if col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.replace(r'[$,]', '', regex=True).str.strip()
            df[col] = pd.to_numeric(df[col], errors='coerce')
    else:
        print(f"Warning: Column '{col}' was expected but not found in the DataFrame. Skipping conversion for this column.")

# Ensure 'Fiscal Year End Date' can be used for sorting and extract year
df['Fiscal Year End Date'] = pd.to_datetime(df['Fiscal Year End Date'])
df['Fiscal_Year'] = df['Fiscal Year End Date'].dt.year

# Sort data for correct pct_change calculation
df.sort_values(by=['Company', 'Fiscal_Year'], inplace=True)

# Calculate year-over-year percentage changes
df['Revenue Growth (%)'] = df.groupby('Company')['Total Revenue'].pct_change() * 100
df['Net Income Growth (%)'] = df.groupby('Company')['Net Income'].pct_change() * 100
df['Assets Growth (%)'] = df.groupby('Company')['Total Assets'].pct_change() * 100
df['Libalities Growth (%)'] = df.groupby('Company')['Total Libalities'].pct_change() * 100
df['Cash Flow from operating activities (%)'] = df.groupby('Company')['Cash Flow from operating activities'].pct_change() * 100

# Calculate Financial Ratios
df['Net Profit Margin'] = (df['Net Income'] / df['Total Revenue']) * 100
df['Debt-to-Assets Ratio'] = (df['Total Libalities'] / df['Total Assets']) * 100

# --- Step 2: Chatbot Design - Define Predefined Queries and Prepare Responses ---

# Get the latest year for easier query formulation
latest_year = df['Fiscal_Year'].max()

# Define a dictionary to store predefined queries and their corresponding response templates.
# These responses will be dynamically filled with actual data.
predefined_queries = {
    # Query 1: Total Revenue for a specific company and year
    f"What was Apple's total revenue in {latest_year}?":
        f"Apple's total revenue in {latest_year} was {{:.2f}} million USD.",
    f"What was Microsoft's total revenue in {latest_year}?":
        f"Microsoft's total revenue in {latest_year} was {{:.2f}} million USD.",
    f"What was Tesla's total revenue in {latest_year}?":
        f"Tesla's total revenue in {latest_year} was {{:.2f}} million USD.",

    # Query 2: Net Income for a specific company and year
    f"What was Apple's net income in {latest_year}?":
        f"Apple's net income in {latest_year} was {{:.2f}} million USD.",
    f"What was Microsoft's net income in {latest_year}?":
        f"Microsoft's net income in {latest_year} was {{:.2f}} million USD.",
    f"What was Tesla's net income in {latest_year}?":
        f"Tesla's net income in {latest_year} was {{:.2f}} million USD.",

    # Query 3: Revenue Growth (%) for a specific company and year (using the second latest year as growth is from previous)
    f"What was Apple's revenue growth in {latest_year-1} to {latest_year}?":
        f"Apple's revenue grew by {{:.2f}}% from {latest_year-1} to {latest_year}.",
    f"What was Microsoft's revenue growth in {latest_year-1} to {latest_year}?":
        f"Microsoft's revenue grew by {{:.2f}}% from {latest_year-1} to {latest_year}.",
    f"What was Tesla's revenue growth in {latest_year-1} to {latest_year}?":
        f"Tesla's revenue grew by {{:.2f}}% from {latest_year-1} to {latest_year}.",

    # Query 4: Net Profit Margin for a specific company and year
    f"What was Apple's net profit margin in {latest_year}?":
        f"Apple's net profit margin in {latest_year} was {{:.2f}}%.",
    f"What was Microsoft's net profit margin in {latest_year}?":
        f"Microsoft's net profit margin in {latest_year} was {{:.2f}}%.",
    f"What was Tesla's net profit margin in {latest_year}?":
        f"Tesla's net profit margin in {latest_year} was {{:.2f}}%.",

    # Query 5: Debt-to-Assets Ratio for a specific company and year
    f"What was Apple's debt-to-assets ratio in {latest_year}?":
        f"Apple's debt-to-assets ratio in {latest_year} was {{:.2f}}%.",
    f"What was Microsoft's debt-to-assets ratio in {latest_year}?":
        f"Microsoft's debt-to-assets ratio in {latest_year} was {{:.2f}}%.",
    f"What was Tesla's debt-to-assets ratio in {latest_year}?":
        f"Tesla's debt-to-assets ratio in {latest_year} was {{:.2f}}%.",
}

# --- Function to retrieve data for responses ---
def get_financial_data(company, fiscal_year, metric_column):
    """
    Retrieves a specific financial metric for a given company and fiscal year.
    Returns the value or None if not found.
    """
    data = df[(df['Company'] == company) & (df['Fiscal_Year'] == fiscal_year)]
    if not data.empty and metric_column in data.columns:
        return data.iloc[0][metric_column]
    return None

# --- Step 3: Basic Chatbot Development - Chatbot Logic ---

def simple_financial_chatbot(user_query):
    """
    A simple rule-based chatbot that responds to predefined financial queries.
    """
    user_query = user_query.strip() # Clean user input

    # Helper to get specific data from the DataFrame for queries
    # Note: For growth queries, the year provided in the question (e.g., 2023)
    # refers to the *end year* of the growth period, so we filter by that.

    # Example for retrieving actual values:
    # Get Microsoft's Total Revenue in 2023
    # msft_revenue_2023 = get_financial_data('Microsoft', latest_year, 'Total Revenue')

    # Implement if-else statements for predefined queries
    if user_query == f"What was Apple's total revenue in {latest_year}?":
        value = get_financial_data('Apple', latest_year, 'Total Revenue')
        if value is not None:
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Microsoft's total revenue in {latest_year}?":
        value = get_financial_data('Microsoft', latest_year, 'Total Revenue')
        if value is not None:
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Tesla's total revenue in {latest_year}?":
        value = get_financial_data('Tesla', latest_year, 'Total Revenue')
        if value is not None:
            return predefined_queries[user_query].format(value)

    elif user_query == f"What was Apple's net income in {latest_year}?":
        value = get_financial_data('Apple', latest_year, 'Net Income')
        if value is not None:
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Microsoft's net income in {latest_year}?":
        value = get_financial_data('Microsoft', latest_year, 'Net Income')
        if value is not None:
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Tesla's net income in {latest_year}?":
        value = get_financial_data('Tesla', latest_year, 'Net Income')
        if value is not None:
            return predefined_queries[user_query].format(value)

    elif user_query == f"What was Apple's revenue growth in {latest_year-1} to {latest_year}?":
        # For growth, we need the metric from the END year of the growth period
        value = get_financial_data('Apple', latest_year, 'Revenue Growth (%)')
        if value is not None and not np.isnan(value):
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Microsoft's revenue growth in {latest_year-1} to {latest_year}?":
        value = get_financial_data('Microsoft', latest_year, 'Revenue Growth (%)')
        if value is not None and not np.isnan(value):
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Tesla's revenue growth in {latest_year-1} to {latest_year}?":
        value = get_financial_data('Tesla', latest_year, 'Revenue Growth (%)')
        if value is not None and not np.isnan(value):
            return predefined_queries[user_query].format(value)

    elif user_query == f"What was Apple's net profit margin in {latest_year}?":
        value = get_financial_data('Apple', latest_year, 'Net Profit Margin')
        if value is not None:
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Microsoft's net profit margin in {latest_year}?":
        value = get_financial_data('Microsoft', latest_year, 'Net Profit Margin')
        if value is not None:
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Tesla's net profit margin in {latest_year}?":
        value = get_financial_data('Tesla', latest_year, 'Net Profit Margin')
        if value is not None:
            return predefined_queries[user_query].format(value)

    elif user_query == f"What was Apple's debt-to-assets ratio in {latest_year}?":
        value = get_financial_data('Apple', latest_year, 'Debt-to-Assets Ratio')
        if value is not None:
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Microsoft's debt-to-assets ratio in {latest_year}?":
        value = get_financial_data('Microsoft', latest_year, 'Debt-to-Assets Ratio')
        if value is not None:
            return predefined_queries[user_query].format(value)
    elif user_query == f"What was Tesla's debt-to-assets ratio in {latest_year}?":
        value = get_financial_data('Tesla', latest_year, 'Debt-to-Assets Ratio')
        if value is not None:
            return predefined_queries[user_query].format(value)

    # --- Error Handling for Unrecognized Queries ---
    else:
        return (
            "Sorry, I can only provide information for specific, predefined queries. "
            "Please try one of the following examples:\n" +
            "\n".join(predefined_queries.keys())
        )

# --- Step 4: Demonstration (Interactive Command-Line Chat) ---
print("\n--- GFC Financial Chatbot Prototype (Rule-Based) ---")
print("Type your query (exactly as listed in the examples) or 'exit' to quit.")
print("\nHere are some example queries you can try:")
for query_example in list(predefined_queries.keys())[:5]: # Show first 5 examples
    print(f"- {query_example}")
print("...") # Indicate there are more examples

while True:
    user_input = input("\nYou: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    
    response = simple_financial_chatbot(user_input)
    print(f"Chatbot: {response}")


Data loaded successfully!

--- GFC Financial Chatbot Prototype (Rule-Based) ---
Type your query (exactly as listed in the examples) or 'exit' to quit.

Here are some example queries you can try:
- What was Apple's total revenue in 2023?
- What was Microsoft's total revenue in 2023?
- What was Tesla's total revenue in 2023?
- What was Apple's net income in 2023?
- What was Microsoft's net income in 2023?
...



You:  what was apple's total revenue in 2023


Chatbot: Sorry, I can only provide information for specific, predefined queries. Please try one of the following examples:
What was Apple's total revenue in 2023?
What was Microsoft's total revenue in 2023?
What was Tesla's total revenue in 2023?
What was Apple's net income in 2023?
What was Microsoft's net income in 2023?
What was Tesla's net income in 2023?
What was Apple's revenue growth in 2022 to 2023?
What was Microsoft's revenue growth in 2022 to 2023?
What was Tesla's revenue growth in 2022 to 2023?
What was Apple's net profit margin in 2023?
What was Microsoft's net profit margin in 2023?
What was Tesla's net profit margin in 2023?
What was Apple's debt-to-assets ratio in 2023?
What was Microsoft's debt-to-assets ratio in 2023?
What was Tesla's debt-to-assets ratio in 2023?



You:  What was Apple's total revenue in 2023?


Chatbot: Apple's total revenue in 2023 was 383285.00 million USD.
